In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torch
import math 
import torch.nn as nn
import random
import itertools 
from tqdm import tqdm
import os
import sys
import time

from colvarsfinder.core import AutoEncoderTask, EigenFunctionTask
from colvarsfinder.nn import AutoEncoder, EigenFunctions 
from colvarsfinder.utils import WeightedTrajectory, integrate_sde_overdamped, calc_weights

###  defines all potentials classes

In [ ]:
def g(a):
    """Gaussian function

    :param a: float, real value
    :return: float, g(a)
    """
    return np.exp(- a ** 2)

class TripleWellPotential:
    def __init__(self, beta, *argv):
        self.beta = beta
        self.dim = 2
        
    def V(self, X):
        assert(type(X) == np.ndarray)
        assert(X.ndim == 2)
        assert(X.shape[1] == 2)
        x = X[:,0]
        y = X[:,1]
        u = g(x) * (g(y - 1/3) - g(y - 5/3))
        v = g(y) * (g(x - 1) + g(x + 1))
        V = 3 * u - 5 * v + 0.2 * (x ** 4) + 0.2 * ((y - 1/3) ** 4)
        return V
    
    def dV_y(self, x, y):
        u = g(x) * (g(y - 1/3) - g(y - 5/3))
        a = g(y) * ((x - 1)*g(x - 1) + (x + 1) * g(x + 1))
        dVx = -6 * x * u + 10 * a + 0.8 * (x ** 3)
        return dVx
    
    def dV_y(self, x, y):
        u = g(x) * ((y - 1/3) * g(y - 1/3) - (y - 5/3) * g(y - 5/3))
        b = g(y) * (g(x - 1) + g(x + 1))
        dVy = -6 * u + 10 * y * b + 0.8 * ((y - 1/3) ** 3)
        return dVy
    
    def gradV(self, X):
        assert(type(X) == np.ndarray)
        assert(X.ndim == 2)
        assert(X.shape[1] == 2)
        return np.column_stack( (self.dV_x(X[:,0], X[:,1]), self.dV_y(X[:,0], X[:,1])) )
        
class TripleWellOneChannelPotential:
    def __init__(self, beta, *argv):
        self.beta = beta
        self.dim = 2
        
    def V(self, X):
        assert(type(X) == np.ndarray)
        assert(X.ndim == 2)
        assert(X.shape[1] == 2)   

        a = + 10 * np.exp(- 25 * X[:,0] ** 2 - (X[:,1] + (1 / 3)) ** 2)
        b = - 3 * np.exp(- X[:,0] ** 2 - (X[:,1] - (5 / 3)) ** 2)
        c = - 5 * np.exp(- X[:,1] ** 2 - (X[:,0] - 1) ** 2)
        d = - 5 * np.exp(- X[:,1] ** 2 - (X[:,0] + 1) ** 2) 
        e = + 0.2 * X[:,0] ** 4 + 0.2 * (X[:,1] - (1 / 3)) ** 4
        V = a + b + c + d + e 
        return V
    
    def dV_x(self, x, y):
        a = - 20 * 25 * x * np.exp(- 25 * x ** 2 - (y + (1 / 3)) ** 2)
        b = + 6 * x * np.exp(- x ** 2 - (y - (5 / 3)) ** 2)
        c = + 10 * (x - 1) * np.exp(- y ** 2 - (x - 1) ** 2)
        d = + 10 * (x + 1) * np.exp(- y ** 2 - (x + 1) ** 2) 
        e = + 0.8 * x ** 3
        dVx = a + b + c + d + e
        return dVx
    
    def dV_y(self, x, y):
        a = - 20 * (y + (1 / 3)) * np.exp(- 25 * x ** 2 - (y + (1 / 3)) ** 2)
        b = + 6 * (y - (5 / 3)) * np.exp(- x ** 2 - (y - (5 / 3)) ** 2)
        c = + 10 * y * np.exp(- y ** 2 - (x - 1) ** 2)
        d = + 10 * y * np.exp(- y ** 2 - (x + 1) ** 2) 
        e = + 0.8 * y ** 3
        dVy = a + b + c + d + e
        return dVy
    
    def gradV(self, X):
        assert(type(X) == np.ndarray)
        assert(X.ndim == 2)
        assert(X.shape[1] == 2)
        return np.column_stack( (self.dV_x(X[:,0], X[:,1]), self.dV_y(X[:,0], X[:,1])) )
        
class DoubleWellPotential:
    def __init__(self, beta, *argv):
        self.beta = beta
        self.dim = 2
        
    def V(self, X):
        assert(type(X) == np.ndarray)
        assert(X.ndim == 2)
        assert(X.shape[1] == 2)
        a = 3 * np.exp(- X[:,0] ** 2 - X[:,1] ** 2)
        b = - 5 * np.exp(- X[:,1] ** 2 - (X[:,0] - 1) ** 2)
        c = - 5 * np.exp(- X[:,1] ** 2 - (X[:,0] + 1) ** 2)
        d = + 0.2 * X[:,0] ** 4 + 0.2 * X[:,1] ** 4
        V = a + b + c + d
        return V
    
    def dV_x(self, x, y):
        a = -6 * x * np.exp(- x ** 2 - y ** 2)
        b = + 10 * (x - 1) * np.exp(- y ** 2 - (x - 1) ** 2)
        d = + 10 * (x + 1) * np.exp(- y ** 2 - (x + 1) ** 2)
        c = + 0.8 * x ** 3
        dVx = a + b + c + d
        return dVx
    
    def dV_y(self, x, y):
        a = - 6 * y * np.exp(- x ** 2 - y ** 2) 
        b = + 10 * y * np.exp(- y ** 2 - (x - 1) ** 2)
        c = + 10 * y * np.exp(- y ** 2 - (x + 1) ** 2)
        d = + 0.8 * y ** 3
        dVy = a + b + c + d
        return dVy
    
    def gradV(self, X):
        assert(type(X) == np.ndarray)
        assert(X.ndim == 2)
        assert(X.shape[1] == 2)
        return np.column_stack( (self.dV_x(X[:,0], X[:,1]), self.dV_y(X[:,0], X[:,1])) )
               
class TripleWellPotAlongCircle :
    def __init__(self, beta, eps):
        self.beta = beta
        self.eps = eps 
        self.dim = 2

    def V(self, x):
      # angle in [-pi, pi] 

      theta = np.arctan2(x[1], x[0])
      # radius
      r = np.sqrt( x[0] * x[0] + x[1] * x[1] )

      v_vec = np.zeros(len(x))
      for idx in range(len(x)) :
          # potential V_1
          if theta > math.pi / 3 : 
            v_vec = (1-(theta * 3 / math.pi- 1.0)**2)**2
          if theta < - math.pi / 3 : 
            v_vec = (1-(theta * 3 / math.pi + 1.0)**2)**2
          if theta > - math.pi / 3 and theta < math.pi / 3:
            v_vec = 3.0 / 5.0 - 2.0 / 5.0 * np.cos(3 * theta)  
      # potential V_2
      v_vec = v_vec * 1.0 + (r - 1)**2 * 1.0 / self.eps + 5.0 * np.exp(-5.0 * r**2) 
      return v_vec

    def gradV(self, x): 
      # angle
      theta = np.arctan2(x[1], x[0])
      # radius
      r = np.sqrt( x[0] * x[0] + x[1] * x[1] )

      if np.fabs(r) < 1e-8: 
          print ("warning: radius is too small! r=%.4e" % r)
      dv1_dangle = 0.0
      # derivative of V_1 w.r.t. angle
      if theta > math.pi / 3: 
        dv1_dangle = 12 / math.pi * (theta * 3 / math.pi - 1) * ((theta * 3 / math.pi- 1.0)**2-1)
      if theta < - math.pi / 3: 
        dv1_dangle = 12 / math.pi * (theta * 3 / math.pi + 1) * ((theta * 3 / math.pi + 1.0)**2-1)
      if theta > -math.pi / 3 and theta < math.pi / 3:
        dv1_dangle = 1.2 * math.sin (3 * theta)
      # derivative of V_2 w.r.t. angle
      dv2_dangle = 0
      # derivative of V_2 w.r.t. radius
      dv2_dr = 2.0 * (r-1.0) / self.eps - 50.0 * r * np.exp(-r**2/0.2)

      return np.array((-(dv1_dangle + dv2_dangle) * x[1] / (r * r)+ dv2_dr * x[0] / r,  (dv1_dangle + dv2_dangle) * x[0] / (r * r)+ dv2_dr * x[1] / r))

class StiffPot : 
    def __init__(self, beta, eps):
        self.beta = beta
        self.eps = eps 
        self.dim = 2

    def V(self, x):
      return (x[:,0]**2 - 1)**2 + 1.0 / self.eps * (x[:,0]**2 + x[:,1] - 1)**2

    def gradV(self, x): 
      return np.column_stack(( 4.0 * x[:,0] * (x[:,0]**2 - 1.0 + 1.0 / self.eps * (x[:,0]**2 + x[:,1] - 1)), 2.0 / self.eps * (x[:,0]**2 + x[:,1] - 1)) )

class UniformPotAlongCircle :
    def __init__(self, beta, eps):
        self.beta = beta
        self.eps = eps 
        self.dim = 2

    def V(self, x):
      return 1.0 / self.eps * (x[:,0]**2 + x[:,1]**2 - 1)**2

    def gradV(self, x): 
      return np.column_stack( (4.0 * x[:,0] / self.eps * (x[:,0]**2 + x[:,1]**2 - 1), 4.0 * x[:,1] / self.eps * (x[:,0]**2 + x[:,1]**2 - 1)) )

class DoubleWellPotAlongCircle :
    def __init__(self, beta, eps):
        self.beta = beta
        self.eps = eps 
        self.dim = 2

    def V(self, x):
        return 2.0 * x[:,1]**2 + 1.0 / self.eps * (x[:,0]**2 + x[:,1]**2 - 1)**2

    def gradV(self, x): 
        return np.column_stack( (4.0 * x[:,0] / self.eps * (x[:,0]**2 + x[:,1]**2 - 1), 4.0 * x[:,1] + 4.0 * x[:,1] / self.eps * (x[:,0]**2 + x[:,1]**2 - 1)) )

### define parameters

In [ ]:
def set_all_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
    random.seed(seed)
    
# x and y domains for each potential
x_domains = [[-2.5, 2.5], [-2.5, 2.5], [-2.5, 2.5], [-3.5, 3.5], [-3.0, 3.0], [-2, 2], [-1.5, 1.5], [-1.5, 1.5] ]
y_domains = [[-1.5, 2.5], [-1.5, 2.5], [-1.5, 2.5], [-3.5, 3.5], [-3.0, 3.0], [-1.8, 1.8], [-1.5, 1.5], [-1.5, 1.5] ]
x0_list = [[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1] ]
v_min_max = [[-4,3], [-4,3], [-4,7], [0.3,8], [0,10], [0,5], [0,5], [0,5] ]

# The names correspond to classes above.
pot_list = [ TripleWellPotential, TripleWellOneChannelPotential, DoubleWellPotential, \
        TripleWellPotAlongCircle, StiffPot, UniformPotAlongCircle, DoubleWellPotAlongCircle ] 

# choose a potential in the pot_list above
pot_id = 3

seed = 30
eps = 0.5
beta = 1.0 
pot = pot_list[pot_id](beta, eps)
pot_name = type(pot).__name__

print ('potential name: %s' % pot_name) 

x_domain = x_domains[pot_id] 
y_domain = y_domains[pot_id] 

# for data generation
delta_t = 0.001
N = 5000000
save = 50

save_fig_to_file = False

# path to store log data
prefix = f"{pot_name}-" 
model_path = os.path.join('checkpoint', prefix + time.strftime("%Y-%m-%d-%H:%M:%S", time.localtime()))
print ('\nLog directory: {}\n'.format(model_path))
        
set_all_seeds(seed)


### visualise the potential

In [ ]:
nx = 100
ny = 150
dx = (x_domain[1] - x_domain[0]) / nx
dy = (y_domain[1] - y_domain[0]) / ny

gridx = np.linspace(x_domain[0], x_domain[1], nx)
gridy = np.linspace(y_domain[0], y_domain[1], ny)
x_plot = np.outer(gridx, np.ones(ny)) 
y_plot = np.outer(gridy, np.ones(nx)).T 

x2d = np.concatenate((x_plot.reshape(nx * ny, 1), y_plot.reshape(nx * ny, 1)), axis=1)

pot_on_grid = np.array([pot.V(x) for x in x2d]).reshape(nx, ny)
print ( "min and max values of potential: (%.4f, %.4f)" % (pot_on_grid.min(), pot_on_grid.max()) )

fig = plt.figure(figsize=(9,3))
ax0 = fig.add_subplot(1, 2, 1, projection='3d')
ax1 = fig.add_subplot(1, 2, 2)
ax0.set_title(pot_name)
print (x_plot.shape, y_plot.shape, pot_on_grid.shape)

ax0.plot_surface(x_plot, y_plot, pot_on_grid, cmap='coolwarm', edgecolor='none')
ax1.pcolormesh(x_plot, y_plot, pot_on_grid, cmap='coolwarm',shading='auto', vmin=v_min_max[pot_id][0], vmax=v_min_max[pot_id][1])

if save_fig_to_file :
    filename = f"{model_path}/{pot_name}.jpg" 
    fig.savefig(filename)
    print ( "potential profiles saved to file: %s" % filename )

In [ ]:
x_0 = np.array(x0_list[pot_id])
traj_filename = 'traj.txt'
csv_filename = 'output.csv'

integrate_sde_overdamped(pot, 1000000, './', X0=x_0, pre_steps=0, step_size=0.001, traj_txt_filename=traj_filename, csv_filename='output.csv', report_interval=100, report_interval_stdout=100000)

calc_weights(csv_filename, pot.beta, pot.beta)

trajectory = np.loadtxt('./traj.txt')

### Plot the trajectory 
fig = plt.figure(figsize=(10,5))
ax0 = fig.add_subplot(1, 2, 1)
ax1 = fig.add_subplot(1, 2, 2)
nx = ny = 200
h = np.histogram2d(trajectory[:,1], trajectory[:,2], bins=[nx, ny], range=[[x_domain[0],x_domain[1]],[y_domain[0],y_domain[1]]])[0]
s = sum(sum(h))
im = ax0.imshow(h.T / (s * dx * dy), origin = "lower", \
                extent=[x_domain[0],x_domain[1],y_domain[0], y_domain[1]], \
                cmap=cm.jet, vmin=0.0, vmax=0.3)
#fig.colorbar(im, cax=cax, cmap=cm.jet)
#ax0.pcolormesh(x_plot, y_plot, pot_on_grid, cmap='coolwarm_r', shading='auto')
ax1.scatter(trajectory[:,1], trajectory[:,2], marker='x')
#ax1.plot(range(len(trajectory[:,0])), trajectory[:,0], label='x coodinate along trajectory')

if save_fig_to_file :
    traj_filename = f"{model_path}/traj_{pot_name}.jpg" 
    fig.savefig(traj_filename)
    print ("trajectory plot saved to file: %s" % traj_filename)

In [ ]:
#### task 1: train autoencoder  

def plot_encoder(self, epoch):
    gridx = np.linspace(x_domain[0], x_domain[1], 100)
    gridy = np.linspace(y_domain[0], y_domain[1], 100)
    x_plot = np.outer(gridx, np.ones(100)) 
    y_plot = np.outer(gridy, np.ones(100)).T 

    x2d = torch.from_numpy(np.concatenate((x_plot.reshape(100 * 100, 1), y_plot.reshape(100 * 100, 1)), axis=1))
    encoder_on_grid = self.model.eval_encoder(x2d).detach().numpy()

    for idx in range(self.k):
        encoder = encoder_on_grid[:,idx].reshape(100,100)
        #print ( "min and max values of %dth dimension of encoder: (%.4f, %.4f)" % (idx, encoder.min(), encoder.max()) )

        fig = plt.figure(figsize=(10,5))
        ax0 = fig.add_subplot(1, 2, 1, projection='3d')
        ax1 = fig.add_subplot(1, 2, 2)  
        ax0.plot_surface(x_plot, y_plot, encoder, cmap='coolwarm', edgecolor='none')
        im = ax1.pcolormesh(x_plot, y_plot, encoder, cmap='coolwarm',shading='auto')
        contours = ax1.contour(x_plot, y_plot, encoder, 20, colors='black')
        ax1.clabel(contours, inline=True, fontsize=8)
        fig.colorbar(im, ax=ax1)
        fig_name = f"{self.model_path}/encoder_{idx}_{epoch}.jpg"
        fig.savefig(fig_name)
        #print ( "encoder profiles saved to file: %s" % fig_name )
        plt.close()
        self.writer.add_image(f'{idx}th dimension of encoder', cv.cvtColor(cv.imread(fig_name), cv.COLOR_BGR2RGB), global_step=epoch, dataformats='HWC')

    
def plot_eigenfunc(self, epoch, cvec):
    gridx = np.linspace(x_domain[0], x_domain[1], 100)
    gridy = np.linspace(y_domain[0], y_domain[1], 100)
    x_plot = np.outer(gridx, np.ones(100)) 
    y_plot = np.outer(gridy, np.ones(100)).T 

    x2d = torch.from_numpy(np.concatenate((x_plot.reshape(100 * 100, 1), y_plot.reshape(100 * 100, 1)), axis=1))
    eigenfuncs_on_grid = self.model(x2d).detach().numpy()

    for idx in range(self.k):
        eigenfunc = eigenfuncs_on_grid[:,cvec[idx]].reshape(100,100)
        #print ( "min and max values of %dth eigenfunc: (%.4f, %.4f)" % (idx, eigenfunc.min(), eigenfunc.max()) )

        fig = plt.figure(figsize=(10,5))
        ax0 = fig.add_subplot(1, 2, 1, projection='3d')
        ax1 = fig.add_subplot(1, 2, 2)            
        ax0.plot_surface(x_plot, y_plot, eigenfunc, cmap='coolwarm', edgecolor='none')
        im=ax1.pcolormesh(x_plot, y_plot, eigenfunc, cmap='coolwarm',shading='auto')
        contours = ax1.contour(x_plot, y_plot, eigenfunc, 20, colors='black')
        ax1.clabel(contours, inline=True, fontsize=8)
        fig.colorbar(im, ax=ax1)

        fig_name = f"{self.model_path}/eigenfunc_{idx}_{epoch}.jpg"
        fig.savefig(fig_name)
        #print ( "%d eigenfunction profile saved to file: %s" % (idx, fig_name) )
        plt.close()
        self.writer.add_image(f'{idx}th eigenfunction', cv.cvtColor(cv.imread(fig_name), cv.COLOR_BGR2RGB), global_step=epoch, dataformats='HWC')

### define parameters and run a training task

In [ ]:
traj = WeightedTrajectory(traj_filename=traj_filename, weight_filename='weights.txt')
pp_layer = torch.nn.Identity()
train_autoencoder = True

batch_size = 500
num_epochs = 200
test_ratio = 0.1
learning_rate = 0.005 
optimizer = 'Adam' # 'SGD'

pp_layer = torch.nn.Identity()

if train_autoencoder:
    e_dims = [2, 20, 20, 20, 1]
    d_dims = [1, 20, 20, 2]
    model = AutoEncoder(e_dims, d_dims)
    model_path = 'checkpoint/autoencoder'
    train_obj = AutoEncoderTask(traj, pp_layer, model, model_path,learning_rate=learning_rate, 
                                batch_size=batch_size, num_epochs=num_epochs,  verbose=False)
else:
    model = EigenFunctions([2, 20, 20, 20, 1], 1)
    model_path = 'checkpoint/eigenfunction'
    beta = 1.0
    diag_coeff = torch.ones(2)
    alpha = 20.0
    eig_weights = [1.0]
    train_obj = EigenFunctionTask(traj, pp_layer, model, model_path, beta, diag_coeff, alpha, eig_weights, 
                                  learning_rate=learning_rate, batch_size=batch_size, num_epochs=num_epochs,  verbose=False)
    
# train autoencoder
train_obj.train()